<a href="https://colab.research.google.com/github/yohoobot/works/blob/main/spt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import pandas as pd
import time

# 🔹 你的 Spotify API Client ID 和 Client Secret
CLIENT_ID = " "
CLIENT_SECRET = " "

# 🔹 Spotify API 认证 URL
AUTH_URL = "https://accounts.spotify.com/api/token"

# 🔹 获取 Access Token（OAuth 认证）
auth_response = requests.post(AUTH_URL, {
    "grant_type": "client_credentials",
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
})

# 🔹 解析 Access Token
auth_response_data = auth_response.json()
ACCESS_TOKEN = auth_response_data.get("access_token")

# 🔹 确保 Token 获取成功
if not ACCESS_TOKEN:
    print("❌ 无法获取 Spotify 访问令牌，请检查 Client ID 和 Client Secret 是否正确。")
    exit()

# 🔹 设置请求头
HEADERS = {
    "Authorization": f"Bearer {ACCESS_TOKEN}"
}

# 🔹 Spotify API 端点
SEARCH_URL = "https://api.spotify.com/v1/search"
PLAYLIST_TRACKS_URL = "https://api.spotify.com/v1/playlists/{playlist_id}/tracks"

# 🔹 关键词列表（适用于餐厅背景音乐）
KEYWORDS = ["restaurant", "cafe", "dining", "coffee shop"]

# 🔹 存储数据
playlist_data = []  # 歌单信息
song_data = []  # 歌曲信息

# 🔹 遍历关键词
for keyword in KEYWORDS:
    print(f"🔍 正在搜索关键词：{keyword} ...")

    # 🔹 构造 API 请求参数
    params = {
        "q": keyword,  # 搜索关键字
        "type": "playlist",  # 搜索类型（歌单）
        "limit": 10  # 返回前 10 个歌单
    }

    # 🔹 发送搜索请求
    response = requests.get(SEARCH_URL, headers=HEADERS, params=params)
    response_data = response.json()

    # 🔹 提取歌单数据
    playlists = response_data.get("playlists", {}).get("items", [])

    # 🔹 解析歌单信息
    for playlist in playlists:
      if playlist:
        playlist_id = playlist.get("id", "未知ID")
        playlist_name = playlist.get("name", "未知歌单")
        playlist_url = playlist.get("external_urls", {}).get("spotify", "无链接")
        creator = playlist.get("owner", {}).get("display_name", "未知创建者")
        track_count = playlist.get("tracks", {}).get("total", 0)

        # 存储歌单信息
        playlist_data.append({
            "关键词": keyword,
            "歌单名称": playlist_name,
            "歌单链接": playlist_url,
            "创建者": creator,
            "歌曲数量": track_count
        })

        print(f"🎵 正在爬取歌单: {playlist_name} ...")

        # 获取歌单内所有歌曲
        track_response = requests.get(PLAYLIST_TRACKS_URL.format(playlist_id=playlist_id), headers=HEADERS)
        track_data = track_response.json()

        # 遍历歌单内歌曲
        tracks = track_data.get("items", [])
        for track in tracks:
            track_info = track.get("track", {})

            # 获取歌曲相关信息
            song_name = track_info.get("name", "未知歌曲")
            song_url = track_info.get("external_urls", {}).get("spotify", "无链接")
            album_name = track_info.get("album", {}).get("name", "未知专辑")
            artist_names = ", ".join([artist["name"] for artist in track_info.get("artists", [])])
            song_popularity = track_info.get("popularity", 0)  # 歌曲受欢迎程度（0-100）

            # Spotify 没有直接提供 "风格"，但可以从专辑名称、艺术家或歌单类别推断
            song_style = keyword  # 这里用关键词作为风格的替代

            # 存储歌曲信息
            song_data.append({
                "歌单名称": playlist_name,
                "歌单链接": playlist_url,
                "歌曲名称": song_name,
                "歌曲链接": song_url,
                "艺术家": artist_names,
                "专辑": album_name,
                "风格": song_style,
                "歌曲受欢迎程度": song_popularity
            })

        # 休息1秒，防止 API 访问过快
        time.sleep(5)

# 🔹 将歌单数据存储到 CSV
df_playlist = pd.DataFrame(playlist_data)
df_playlist.to_csv("spotify_playlist_data.csv", encoding="utf-8-sig", index=False)

# 🔹 将歌曲数据存储到 CSV
df_songs = pd.DataFrame(song_data)
df_songs.to_csv("spotify_songs_data.csv", encoding="utf-8-sig", index=False)

print("✅ Spotify 歌单 & 歌曲数据爬取完成！")


🔍 正在搜索关键词：restaurant ...
🎵 正在爬取歌单: Restaurant Music 2025 ...
🎵 正在爬取歌单: restaurant lounge vibes but in a cool, chill, funky way ...
🎵 正在爬取歌单: RESTAURANT MUSIC 2025 🍽  chill covers & dinner songs ...
🎵 正在爬取歌单: Upbeat clean restaurant 2.0 ...
🎵 正在爬取歌单: fancy restaurant music ...
🎵 正在爬取歌单: Clean Restaurant Music 2023 / 2024 (2) ...
🎵 正在爬取歌单: Restaurant Lounge Music 2025 🍸 Background Music 🍸 Dinner Chill ...
🔍 正在搜索关键词：cafe ...
🎵 正在爬取歌单: Cozy Coffee Shop ☕ Jazz in the Background ...
🎵 正在爬取歌单: lofi Jazz Cafe 🎷Study Beats & Chill Vibes ...
🎵 正在爬取歌单: Café Music 2025 ☕ Chill Vibes -  Coffee Lounge - Morning Chill - Relax  ...
🎵 正在爬取歌单: CAFE TACVBA EXITOS ...
🔍 正在搜索关键词：dining ...
🎵 正在爬取歌单: fancy restaurant music ...
🎵 正在爬取歌单: SINING - Dionela Jay R ...
🎵 正在爬取歌单: PIANO COVERS 2025 🎹 relaxing versions of popular songs ...
🎵 正在爬取歌单: dilaw, sining, hoodie, museo, kape, sunset.. ikaw nalang kulang ...
🎵 正在爬取歌单: Fine Dining Music ...
🎵 正在爬取歌单: INSTRUMENTAL COVERS 2025 ✨ popular songs ...
🎵 正在爬取歌单: Dinn